In [ ]:
import cv2
import numpy as np
from PIL import Image
from IPython.display import display

In [ ]:
img_path  = '../pipeline_optimization_dataset/1f5GmaeVFWa2mqhh2V8VucQ1du9vyqllF__Einbauküche_.jpg'   
mask_path = '../results_pipeline_optimization_dataset/results_final_optimization_run/segmentation_masks_bw_dilation0/1f5GmaeVFWa2mqhh2V8VucQ1du9vyqllF__Einbauküche_.png'   

search_px = 140     
delta = 2       
min_touch = 10       
safety_px = 2  

## Defines the `expand_mask_shadow_relative` function, which:
- Computes a ring of interest around the existing mask  
- Finds the local median brightness (“wall” level)  
- Selects darker pixels (potential shadows) within that ring  
- Keeps only shadow regions that touch the object border  
- Optionally smooths the result with a small dilation 

In [ ]:
def expand_mask_shadow_relative(img_gray: np.ndarray, mask_bool: np.ndarray, search_px: int = 80, delta: int = 8, min_touch: int = 3, safety_px: int = 2) -> np.ndarray:

    mask_u8 = mask_bool.astype(np.uint8) * 255
    
    dist = cv2.distanceTransform(255 - mask_u8, cv2.DIST_L2, 5)
    roi  = dist <= search_px
    if not roi.any():
        return mask_bool.copy()
    
    wall_med = np.median(img_gray[roi])
    
    shadow_cand = (img_gray < wall_med - delta) & roi
    
    border = cv2.dilate(mask_u8, None, iterations=min_touch) > 0
    _, lbl, _, _ = cv2.connectedComponentsWithStats(
        shadow_cand.astype(np.uint8), connectivity=4
    )
    touching_lbls = np.setdiff1d(np.unique(lbl[border]), 0)
    shadow_ok = np.isin(lbl, touching_lbls)
    
    merged = mask_bool | shadow_ok
    if safety_px:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (safety_px*2+1,)*2)
        merged = cv2.dilate(merged.astype(np.uint8), k) > 0
    return merged

In [ ]:
def overlay_mask(img_bgr: np.ndarray, mask_bool: np.ndarray, alpha: float = 0.45) -> np.ndarray: 
    overlay = img_bgr.copy()
    overlay[mask_bool] = (0, 0, 255)          
    blended = cv2.addWeighted(overlay, alpha, img_bgr, 1 - alpha, 0)
    return blended[..., ::-1]  

In [ ]:
img_bgr   = cv2.imread(img_path, cv2.IMREAD_COLOR)
if img_bgr is None:
    raise FileNotFoundError(f"Cannot open image: {img_path}")

img_gray  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

mask_raw  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
if mask_raw is None:
    raise FileNotFoundError(f"Cannot open mask:  {mask_path}")

mask_bool = mask_raw > 0          

print('Image shape :', img_bgr.shape)
print('Pixels in original mask:', mask_bool.sum())

In [ ]:
expanded_mask = expand_mask_shadow_relative(
    img_gray, mask_bool,
    search_px=search_px,
    delta=delta,
    min_touch=min_touch,
    safety_px=safety_px
)

print('Pixels in expanded mask:', expanded_mask.sum())


preview_rgb = overlay_mask(img_bgr, expanded_mask)

display(Image.fromarray(preview_rgb))